In [64]:
%load_ext autoreload
%autoreload 2

In [1]:
# Dependencies
import requests
import pymongo
import dns
import datetime
from datetime import datetime
from dateutil import parser
import pytz
from django.utils import timezone
from time import sleep
import random
import json
import config
import requests
from flask_cors import CORS

# Project defined dependencies
import config
from pprint import pprint
# import cl_parser


In [2]:
# Initialize PyMongo to work with MongoDBs
client = pymongo.MongoClient(config.mongo_conn)

In [3]:
# Define database and collection
db = client.getitdone_db
SR_collection = db.sandiego
summary_collection = db.summary_counts

In [68]:
for SR in SR_collection.find():
    print(SR['service_request_id'])
    break

3190603


In [69]:
summary_collection.count_documents({'year':2021})

1

In [70]:
summary_collection.count_documents({'year':2021})

1

In [71]:
for summary_obj in summary_collection.find():
    print(summary_obj)
    break

{'_id': ObjectId('606bc96f90ae90d638d6c766'), 'year': 2019, 'summary': {'01-January': 19285, '02-February': 21736, '03-March': 23412, '04-April': 23070, '05-May': 24885, '06-June': 25819, '07-July': 33100, '08-August': 34470, '09-September': 28657, '10-October': 27163, '11-November': 23375, '12-December': 24233}, 'chart_url': 'https://charts.mongodb.com/charts-dev-exqny/embed/charts?id=6cdc5a0f-6300-48cb-95b2-27fae819b23d&autoRefresh=1800&theme=light', 'chart_url_council_dist': 'https://charts.mongodb.com/charts-dev-exqny/embed/charts?id=f5da86cd-f5b9-41ef-a58c-9b9d308d37c5&autoRefresh=1800&theme=light', 'total_closed': 258716, 'percent_closed': 0.836713507220129, 'total_requested': 309205, 'average_case_age_days': 51.116042754806685}


In [ ]:
currentMonth = datetime.now().month
print(currentMonth)
type(currentMonth)

In [ ]:

local = pytz.timezone("America/Los_Angeles")
dt_start = datetime.strptime("2021-" + str(currentMonth) + "-1 00:00:00", "%Y-%m-%d %H:%M:%S")
dt_start_local = local.localize(dt_start, is_dst=None)
dt_start_utc = dt_start_local.astimezone(pytz.utc)

dt_end = datetime.strptime("2021-" + str(currentMonth + 1) + "-1 00:00:00", "%Y-%m-%d %H:%M:%S")
dt_end_local = local.localize(dt_end, is_dst=None)
dt_end_utc = dt_end_local.astimezone(pytz.utc)

 
print(dt_start_utc, dt_end_utc)

In [75]:
year = 2020
local = pytz.timezone("America/Los_Angeles")
dt_start = datetime.strptime(str(year) + "-1-1 00:00:00", "%Y-%m-%d %H:%M:%S")
dt_start_local = local.localize(dt_start, is_dst=None)
dt_start_utc = dt_start_local.astimezone(pytz.utc)

dt_end = datetime.strptime(str(year + 1) +"-1-1 00:00:00", "%Y-%m-%d %H:%M:%S")
dt_end_local = local.localize(dt_end, is_dst=None)
dt_end_utc = dt_end_local.astimezone(pytz.utc)

filter_requested ={
    'date_requested': {
        '$gte': dt_start_utc, 
        '$lt': dt_end_utc
    }
}
filter_closed ={
    'date_requested': {
        '$gte': dt_start_utc, 
        '$lt': dt_end_utc
    }, 'status':"Closed"
}


total_requested = SR_collection.count_documents(filter=filter_requested)

sum_case_age_days = 0
for SR in SR_collection.find(filter=filter_requested):
    if 'case_age_days' in SR:
        sum_case_age_days += int(SR["case_age_days"])

total_closed = SR_collection.count_documents(filter=filter_closed)



In [76]:
print(dt_start_utc, dt_end_utc)
print(total_requested, total_closed)
print(sum_case_age_days)

2020-01-01 08:00:00+00:00 2021-01-01 08:00:00+00:00
286092 235298
8045454


In [77]:
            update_doc = summary_collection.find_one_and_update(
                {'year' : year},
                {'$set':
                    {
                        "total_requested":int(total_requested),
                        "total_closed":int(total_closed),
                        "percent_closed": float(total_closed/total_requested),
                        "average_case_age_days": float(sum_case_age_days/total_requested)
                        
                    }
                    
                },upsert=True
            )

In [6]:
def updateDistricts(district):
    filter_requested ={'council_district': district}
    filter_closed ={'council_district': district, 'status':"Closed"}
    
    total_requested = SR_collection.count_documents(filter=filter_requested)

    sum_case_age_days = 0
    for SR in SR_collection.find(filter=filter_requested):
        if 'case_age_days' in SR:
            sum_case_age_days += int(SR["case_age_days"])

    total_closed = SR_collection.count_documents(filter=filter_closed)
    
    update_doc = summary_collection.find_one_and_update(
                {'district' : district},
                {'$set':
                    {
                        "total_requested":int(total_requested),
                        "total_closed":int(total_closed),
                        "percent_closed": float(total_closed/total_requested),
                        "average_case_age_days": float(sum_case_age_days/total_requested)
                        
                    }
                    
                },upsert=True
            )
    
districts = [1,2,3,4,5,6,7,8,9]
for district in districts:
    updateDistricts(district)

In [ ]:
filter={
    'date_requested': {
        '$gte': dt_start_utc, 
        '$lt': dt_end_utc
    }
}

result = client['getitdone_db']['sandiego'].find(filter=filter)

In [ ]:
i = 0
date_requested_count = {}  
date_closed_count = {}

for SR in result:
    
    if 'date_requested' in SR:            
        date_requested_month = SR['date_requested'].month     
        if (date_requested_month in date_requested_count):
            date_requested_count[date_requested_month] += 1        
        else:
            date_requested_count[date_requested_month] = 1            
    if 'date_closed' in SR:
        date_closed_month = SR['date_closed'].month     
        if (date_closed_month in date_closed_count):
            date_closed_count[date_closed_month] += 1        
        else:
            date_closed_count[date_closed_month] = 1 
    
    # SR['case_age_days']
    # SR['service_name']
    # SR['case_record_type']    
    # SR['status']
    # SR['lat']
    # SR['lng']
    # SR['street_address']
    # SR['council_district']
    # SR['comm_plan_code']
    # SR['comm_plan_name']
    # SR['case_origin']
    # SR['public_description']   

    # i += 1
    # if i > 5:
    #     break

In [ ]:
date_requested_count


In [ ]:
date_closed_count

In [ ]:
def get_sd_api_data(sd_api_url):
    print("import SR from get it done api")        
    response = requests.get(sd_api_url)
    response_json = response.json()  

    count_insert_SR = 0
    count_update_SR = 0

    for SR in response_json:
        public_description = ""
        media_url = ""

        # Get the data from the results
        service_request_id = int(SR["service_request_id"])
        date_requested_string = SR["requested_datetime"] # or any date sting of differing formats.
        date_requested = parser.parse(date_requested_string)
        updated_datetime_string = SR["updated_datetime"] # or any date sting of differing formats.
        updated_datetime = parser.parse(updated_datetime_string)        
        
        if "description" in SR:
            public_description = SR["description"]
        if 'media_url' in SR:
            media_url = SR["media_url"]

        if SR_collection.count_documents({'service_request_id': service_request_id}) == 0:
            print("Insert new service request! ")            
            insert_doc = { 
                "service_request_id": service_request_id,
                "date_requested": date_requested,
                "date_updated": updated_datetime,
                "status": SR["status"],
                "service_code": SR["service_code"],
                "service_name": SR["service_name"],
                "public_description": public_description,
                "street_address": SR["address"],
                "lat": float(SR["lat"]),
                "lng": float(SR["long"]),
                "media_url": media_url
                }
            doc = SR_collection.insert_one(insert_doc)
            count_insert_SR += 1
            # if config.debug:
            #     print(doc)

        else: 
            print(f"SR {service_request_id} already exists, update existing service request!")
            update_doc = SR_collection.find_one_and_update(
                {'service_request_id' : service_request_id},
                {'$set':
                    {
                        "date_requested": date_requested,
                        "date_updated": updated_datetime,
                        "status": SR["status"],
                        "service_code": SR["service_code"],
                        "service_name": SR["service_name"],
                        "public_description": public_description,
                        "street_address": SR["address"],
                        "lat": float(SR["lat"]),
                        "lng": float(SR["long"]),
                        "media_url": media_url
                    }
                    
                },upsert=True
            )
            count_update_SR += 1
            # if config.debug:
            #     print(update_doc)

    return f"{str(count_insert_SR)} SR created. {str(count_update_SR)} SR updated."

            

In [ ]:
summary = get_sd_api_data("http://san-diego.spotreporters.com/open311/v2/requests.json")

In [ ]:
print(summary)